# Gradnja z modeli Mistral

## Uvod

Ta lekcija bo zajemala:
- Raziskovanje različnih modelov Mistral
- Razumevanje primerov uporabe in scenarijev za vsak model
- Primeri kode prikazujejo edinstvene značilnosti vsakega modela.


## Modeli Mistral

V tej lekciji bomo raziskali 3 različne modele Mistral:  
**Mistral Large**, **Mistral Small** in **Mistral Nemo**.

Vsak od teh modelov je brezplačno na voljo na Github Model marketplace. Koda v tem zvezku bo uporabljala te modele za izvajanje kode. Tukaj so podrobnejše informacije o uporabi Github Modelov za [prototipiranje z AI modeli](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 je trenutno vodilni model podjetja Mistral in je zasnovan za poslovno uporabo.

Model je nadgradnja izvirnega Mistral Large z zagotavljanjem
- Večjega kontekstnega okna - 128k proti 32k
- Boljših zmogljivosti pri matematičnih in programerskih nalogah - povprečna natančnost 76,9 % proti 60,4 %
- Povečane večjezične zmogljivosti - jeziki vključujejo: angleščino, francoščino, nemščino, španščino, italijanščino, portugalščino, nizozemščino, ruščino, kitajščino, japonščino, korejščino, arabščino in hindijščino.

S temi funkcijami Mistral Large izstopa pri
- *Generiranju z nadgradnjo z iskanjem (RAG)* - zaradi večjega kontekstnega okna
- *Klicanju funkcij* - ta model ima nativno klicanje funkcij, kar omogoča integracijo z zunanjimi orodji in API-ji. Ti klici se lahko izvajajo vzporedno ali zaporedno eden za drugim.
- *Generiranju kode* - ta model izstopa pri generiranju v Pythonu, Javi, TypeScriptu in C++.


### Primer RAG z uporabo Mistral Large 2


V tem primeru uporabljamo Mistral Large 2 za izvajanje vzorca RAG nad besedilnim dokumentom. Vprašanje je napisano v korejščini in sprašuje o dejavnostih avtorja pred fakulteto.

Uporablja Cohere Embeddings Model za ustvarjanje vdelav besedilnega dokumenta in vprašanja. Za ta primer uporablja Python paket faiss kot vektorsko bazo.

Poziv, poslan modelu Mistral, vključuje tako vprašanja kot pridobljene dele, ki so podobni vprašanju. Model nato zagotovi odgovor v naravnem jeziku.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small je še en model v družini modelov Mistral v kategoriji premier/enterprise. Kot ime pove, je ta model Majhen Jezikovni Model (SLM). Prednosti uporabe Mistral Small so: 
- Prihranek stroškov v primerjavi z Mistral LLM-ji, kot so Mistral Large in NeMo - 80 % znižanje cene
- Nizka zakasnitev - hitrejši odziv v primerjavi z Mistral LLM-ji
- Prilagodljiv - lahko se namesti v različnih okoljih z manj omejitvami glede potrebnih virov. 


Mistral Small je odličen za: 
- Naloge, ki temeljijo na besedilu, kot so povzemanje, analiza sentimenta in prevajanje. 
- Aplikacije, kjer so pogoste zahteve zaradi njegove stroškovne učinkovitosti 
- Naloge z nizko zakasnitvijo, kot so pregled in predlogi kode 


## Primerjava Mistral Small in Mistral Large

Za prikaz razlik v zakasnitvi med Mistral Small in Large zaženite spodnje celice.

Videli bi morali razliko v časih odziva med 3 in 5 sekundami. Prav tako opazite dolžino in slog odzivov na isti poziv.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

V primerjavi z drugima dvema modeloma, obravnavanima v tej lekciji, je Mistral NeMo edini brezplačni model z licenco Apache2.

Šteje se za nadgradnjo prej odprtokodnega LLM iz Mistrala, Mistral 7B.

Nekatere druge značilnosti modela NeMo so:

- *Učinkovitejša tokenizacija:* Ta model uporablja tokenizator Tekken namesto bolj pogosto uporabljenega tiktoken. To omogoča boljšo zmogljivost za več jezikov in kode.

- *Fino nastavljanje:* Osnovni model je na voljo za fino nastavljanje. To omogoča večjo prilagodljivost za primere uporabe, kjer je fino nastavljanje morda potrebno.

- *Nativno klicanje funkcij* - Tako kot Mistral Large je bil ta model usposobljen za klicanje funkcij. To ga naredi edinstvenega kot enega izmed prvih odprtokodnih modelov, ki to omogočajo.


## Mistral NeMo

V primerjavi z drugima dvema modeloma, obravnavanima v tej lekciji, je Mistral NeMo edini brezplačni model z licenco Apache2.

Obravnava se ga kot nadgradnjo prej odprtokodnega LLM iz Mistral, Mistral 7B.

Nekatere druge značilnosti modela NeMo so:

- *Učinkovitejša tokenizacija:* Ta model uporablja tokenizator Tekken namesto bolj pogosto uporabljenega tiktoken. To omogoča boljšo zmogljivost pri več jezikih in kodi.

- *Fino nastavljanje:* Osnovni model je na voljo za fino nastavljanje. To omogoča večjo prilagodljivost za primere uporabe, kjer je fino nastavljanje morda potrebno.

- *Nativno klicanje funkcij* - Tako kot Mistral Large je bil ta model usposobljen za klicanje funkcij. To ga naredi edinstvenega kot enega prvih odprtokodnih modelov, ki to omogočajo.


### Primerjava tokenizatorjev

V tem vzorcu bomo pogledali, kako Mistral NeMo obravnava tokenizacijo v primerjavi z Mistral Large.

Oba vzorca vzameta isti poziv, vendar bi morali videti, da NeMo vrne manj tokenov v primerjavi z Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Učenje se tukaj ne konča, nadaljujte potovanje

Po zaključku te lekcije si oglejte našo [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), da nadaljujete z nadgrajevanjem svojega znanja o generativni umetni inteligenci!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas opozarjamo, da avtomatizirani prevodi lahko vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Za morebitna nesporazume ali napačne interpretacije, ki izhajajo iz uporabe tega prevoda, ne odgovarjamo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
